In [1]:
# Importing necessary libraries
import os  # Handling file and directory operations
import requests  # Sending HTTP requests to web pages
import pandas as pd  # Handling tabular data
from bs4 import BeautifulSoup  # Parsing HTML content
from tqdm import tqdm  # Displaying progress bars
from urllib.parse import urljoin  # Joining relative URLs with base
from PIL import Image  # Opening and saving image files
from io import BytesIO  # Handling image bytes in memory
import time  # Adding delays between requests
import spacy  # Performing NLP for filename shortening
import re  # Applying regex for filename cleaning
import random  # Generating random numbers for delays

In [2]:
# Defining category-wise Amazon search URLs
search_urls = {
    "Clothing": "https://www.amazon.co.uk/s?k=clothing",
    "Footwear": "https://www.amazon.co.uk/s?k=footwear",
    "Accessories": "https://www.amazon.co.uk/s?k=accessories",
    "Furniture": "https://www.amazon.co.uk/s?k=furniture",
    "Food Items": "https://www.amazon.co.uk/s?k=food+items",
    "Household Essentials": "https://www.amazon.co.uk/s?k=household+essentials",
    "Personal Care": "https://www.amazon.co.uk/s?k=personal+care",
    "Lighting": "https://www.amazon.co.uk/s?k=lighting",
    "Stationery": "https://www.amazon.co.uk/s?k=stationery",
    "Grocery": "https://www.amazon.co.uk/s?k=grocery"
}

In [ ]:
# Defining headers to mimic a real browser and reduce blocking risk
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

In [4]:
# Collecting product links by paginating through Amazon search results
def get_product_links(search_url, required_products=100):
    
    product_urls = set()  # Initializing a set to store unique product URLs
    current_page = 1  # Starting from page 1

    while len(product_urls) < required_products:

        # Creating paginated URL by appending page number
        paginated_url = f"{search_url}&page={current_page}"
        print(f"🔄 Fetching page {current_page}: {paginated_url}")
        
        # Sending request to Amazon with headers
        response = requests.get(paginated_url, headers=HEADERS)

        # Checking if response is successful
        if response.status_code != 200:
            print(f"❌ Failed to fetch page {current_page}")
            break

        # Parsing HTML content
        soup = BeautifulSoup(response.content, "html.parser")

        # Finding all product links on the current page
        links = soup.find_all("a", class_="a-link-normal s-no-outline")

        # Iterating through each link
        for link in links:
            href = link.get("href")
            if href:
                # Constructing full product URL
                full_url = urljoin("https://www.amazon.co.uk", href)
                product_urls.add(full_url)

            # Stopping if required number of URLs is reached
            if len(product_urls) >= required_products:
                break

        # Breaking the loop if no links are found (i.e., last page)
        if not links:
            print("⚠️ No more product links found.")
            break

        current_page += 1  # Moving to next page
        time.sleep(random.uniform(1.5, 3.0))  # Adding delay to avoid rate-limiting

    # Returning only the required number of product URLs
    return list(product_urls)[:required_products]

In [5]:
# Scraping title, price, image URL and description from a product page
def scrape_product_details(product_url):
    
    # Sending request to product page
    response = requests.get(product_url, headers=HEADERS)

    # Returning if response fails
    if response.status_code != 200:
        print(f"❌ Failed to fetch product page: {product_url}")
        return None

    # Parsing the HTML content
    soup = BeautifulSoup(response.content, "html.parser")

    # Extracting the product title
    title_element = soup.find("span", attrs={"id": "productTitle"})
    title = title_element.text.strip() if title_element else "Unknown"

    # Extracting the product price
    price = "N/A"
    try:
        price_element = soup.find("div", attrs={"class": "a-section a-spacing-none aok-align-center aok-relative"})
        if price_element:
            price = price_element.find("span", attrs={"class": "aok-offscreen"}).string.strip()
    except AttributeError:
        print(f"⚠️ Price not found for {title}, using default 'N/A'.")

    # Extracting one product image URL
    image_tag = soup.find("img", class_="a-dynamic-image")
    image_url = image_tag.get("data-old-hires") or image_tag.get("src") if image_tag else None

    # Extracting the product description
    description = "No description available"
    try:
        description_list = soup.find("ul", class_="a-unordered-list a-vertical a-spacing-mini")
        if not description_list:
            description_list = soup.find("ul", class_="a-unordered-list a-vertical a-spacing-small")

        if description_list:
            bullet_points = description_list.find_all("span", class_="a-list-item")
            description = " ".join([item.text.strip() for item in bullet_points if item.text.strip()])
    except AttributeError:
        print(f"⚠️ Description not found for {title}, using default.")

    # Returning collected product details
    return {
        "Title": title,
        "Price": price,
        "Product URL": product_url,
        "Image URL": image_url,
        "Description": description
    }

In [6]:
# Loading spaCy NLP model for smart filename shortening
nlp = spacy.load("en_core_web_sm")

# Creating short filenames using extracted keywords from product name
def smart_shorten_name(product_name, max_words=5):
    
    # Processing the product name using spaCy NLP
    doc = nlp(product_name)

    # Extracting nouns and proper nouns as keywords
    keywords = [token.text for token in doc if token.pos_ in {"NOUN", "PROPN"}]

    # Using adjectives if no nouns are found
    if not keywords:
        keywords = [token.text for token in doc if token.pos_ in {"ADJ"}]

    # Falling back to first 3 words if still empty
    if not keywords:
        keywords = product_name.split()[:3]

    # Joining keywords with underscores
    short_name = "_".join(keywords[:max_words])

    # Returning cleaned name or fallback to original with underscores
    return short_name if short_name.strip() else product_name.replace(" ", "_")

In [7]:
# Saving one image per product in a category-specific folder with serial numbers
def save_image(category, product_name, image_url, serial_number):
    
    # Creating category folder if not already present
    category_folder = os.path.join("product_images", category)
    os.makedirs(category_folder, exist_ok=True)

    try:
        # Downloading the image from the URL
        response = requests.get(image_url, headers=HEADERS)
        response.raise_for_status()

        # Opening image in memory
        image = Image.open(BytesIO(response.content))

        # Converting to RGB if image has transparency or palette
        if image.mode in ("P", "RGBA"):
            image = image.convert("RGB")

        # Generating a cleaned, NLP-based short filename
        cleaned_name = smart_shorten_name(product_name)
        cleaned_name = re.sub(r'[^a-zA-Z0-9_]', '', cleaned_name)  # Removing special characters

        # Skipping if filename is empty
        if not cleaned_name.strip():
            print(f"⚠️ Skipping product '{product_name}' (no valid filename generated).")
            return None

        # Formatting filename with serial number
        serial_str = f"{serial_number:02d}"
        image_filename = f"{serial_str}_{cleaned_name}.jpg"
        image_path = os.path.join(category_folder, image_filename)
        image.save(image_path, "JPEG")
        print(f"Saved: {image_path}")
        return image_path

    except Exception as e:
        print(f"❌ Failed to download image: {e}")
        return None

In [8]:
# Scraping 100 unique products for each category and saving images
# Loading already-saved CSV if it exists
existing_categories = set()

if os.path.exists("product_data.csv"):
    existing_df = pd.read_csv("product_data.csv")
    existing_categories = set(existing_df['Category'].unique())
    all_products = existing_df.to_dict(orient="records")
    print(f"🔁 Resuming scraping. Skipping already-scraped categories: {existing_categories}")
else:
    all_products = []  # Initializing final product list

# Iterating through all categories
for category, search_url in search_urls.items():
    if category in existing_categories:
        print(f"⏭️ Skipping {category} (already scraped)")
        continue

    print(f"\n🔍 Scraping category: {category}")

    # Collecting up to 100 product links via pagination
    product_links = get_product_links(search_url, required_products=150)
    valid_products = []
    unique_product_titles = set()

    # Looping through collected product URLs
    for product_url in tqdm(product_links, desc=f"Scraping {category} Products"):
        if len(valid_products) >= 100:
            break

        # Scraping details from product page
        product_data = scrape_product_details(product_url)

        # Checking if data and image are valid
        if product_data and product_data["Image URL"]:
            product_title = product_data["Title"]

            # Generating NLP-shortened name
            shortened_name = smart_shorten_name(product_title)

            # Skipping duplicates based on shortened name
            if shortened_name in unique_product_titles:
                print(f"⚠️ Skipping duplicate product: {shortened_name}")
                continue  

            # Saving the product image
            serial_number = len(valid_products) + 1
            saved_image = save_image(category, shortened_name, product_data["Image URL"], serial_number)


            # Adding product to the list if image is saved successfully
            if saved_image:
                product_data["Title"] = shortened_name  
                product_data["Saved Image"] = saved_image
                product_data["Category"] = category

                valid_products.append(product_data)
                unique_product_titles.add(shortened_name)  

    # Adding all valid products from this category to the master list
    all_products.extend(valid_products)
    print(f"✅ Collected {len(valid_products)} unique products for {category}")

    # Saving to CSV after each category
    columns = ["Category", "Title", "Price", "Product URL", "Image URL", "Saved Image", "Description"]
    df_partial = pd.DataFrame(all_products)[columns]
    df_partial.to_csv("product_data.csv", index=False)
    print(f"✅ Saved {len(df_partial)} total products so far to product_data.csv")
    
    time.sleep(5)  # Adding delay between categories

🔁 Resuming scraping. Skipping already-scraped categories: {'Lighting', 'Clothing', 'Footwear', 'Furniture', 'Food Items', 'Stationery', 'Accessories', 'Household Essentials', 'Personal Care'}
⏭️ Skipping Clothing (already scraped)
⏭️ Skipping Footwear (already scraped)
⏭️ Skipping Accessories (already scraped)
⏭️ Skipping Furniture (already scraped)
⏭️ Skipping Food Items (already scraped)
⏭️ Skipping Household Essentials (already scraped)
⏭️ Skipping Personal Care (already scraped)
⏭️ Skipping Lighting (already scraped)
⏭️ Skipping Stationery (already scraped)

🔍 Scraping category: Grocery
🔄 Fetching page 1: https://www.amazon.co.uk/s?k=grocery&page=1
🔄 Fetching page 2: https://www.amazon.co.uk/s?k=grocery&page=2
🔄 Fetching page 3: https://www.amazon.co.uk/s?k=grocery&page=3


Scraping Grocery Products:   1%|          | 1/150 [00:05<12:35,  5.07s/it]

Saved: product_images\Grocery\01_Mr_Muscle_Drain_Unblocker_Sink.jpg


Scraping Grocery Products:   1%|▏         | 2/150 [00:08<10:12,  4.14s/it]

Saved: product_images\Grocery\02_La_Tazza_Doro_Coffee_Beans.jpg


...

Scraping Grocery Products:  75%|███████▌  | 113/150 [07:38<02:36,  4.23s/it]

Saved: product_images\Grocery\99_Super_Stix_Ube_Flavour_Wafer.jpg


Scraping Grocery Products:  76%|███████▌  | 114/150 [07:43<02:26,  4.06s/it]

Saved: product_images\Grocery\100_Nestle_Pure_Life_Spring_Water.jpg
✅ Collected 100 unique products for Grocery


✅ Saved 1000 total products so far to product_data.csv
